In [1]:
from gym import Env
from gym.spaces import Discrete, box
import numpy as np
import random

In [2]:
class DeliveryQAgent(QAgent):

    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.reset_memory()

    def act(self,s):

        # Get Q Vector
        q = np.copy(self.Q[s,:])

        # Avoid already visited states
        q[self.states_memory] = -np.inf

        if np.random.rand() > self.epsilon:
            a = np.argmax(q)
        else:
            a = np.random.choice([x for x in range(self.actions_size) if x not in self.states_memory])

        return a


    def remember_state(self,s):
        self.states_memory.append(s)

    def reset_memory(self):
        self.states_memory = []

NameError: name 'QAgent' is not defined

In [3]:
## Create the TSP Environment
import numpy as np
import gym
from gym import spaces
import pygame

from scipy.spatial.distance import cdist
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")

class TSPEnvironment(gym.Env):
    def __init__ (self, n_stops = 100):
        print(f"TSP-Environment initialized with {n_stops} random stops")

        # Initialization
        #Number of stops
        self.n_stops = n_stops
        #Coordinates of stops
        self.xy = []
        
        self.action_space = spaces.Discrete(n_stops)
        self.observation_space = spaces.Discrete(n_stops)
        self.episode_length = n_stops
        self._visitedStops = []

        #set starting point (state)
        
        #Generate stops
        self._generate_stops()
        self._generate_q_values()
                
        

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):
        return {"distance": np.linalg.normal(self._agent_location - self._target_location, ord=1)} 

    def _generate_stops(self):
        self.xy = (np.random.rand(100,2)*100).round(2)
        self.x=self.xy[:,0]
        self.y=self.xy[:,1]

        #print(f'genrated stops xy: {self.xy}')
        
        #pick random StartPoint
        self._visitedStops.append(np.random.randint(0,self.n_stops))
        print(f'Starting Point: {self._visitedStops}')


    def _generate_q_values(self,box_size = 0.2):
        self.q_stops = -cdist(self.xy,self.xy,'euclidean').round(2)
        #print(f'Distance: \n {self.q_stops}')


    def render(self):
        pass

    #Resets StartingPoint
    def reset(self):
        self._visitedStops = []

        first_stop = np.random.randint(self.n_stops)
        self._visitedStops.append(first_stop)
        return first_stop

    def step(self,destination):
        #Get reward for such a move
        reward = self._get_reward(self._get_state(), destination)

        #set new position of agent
        #self.state = destination
        if(destination not in self._visitedStops):
            self._visitedStops.append(destination)

        print(f'Visited Stops: {self._visitedStops}')

        done = len(self._visitedStops) == self.n_stops
        if(done):
            reward+=200
            
        info = {}

        #print(f'Agent position: {self._get_state()}')

        return self._get_state(), reward, done, info


    #return stops[-1]
    #Gibt die aktuelle Position des Agenten zurück
    def _get_state(self):
        if( len(self._visitedStops)>0):
            return self._visitedStops[-1]
        else:
            return 0 

    def _get_reward(self, state, destination):
        return self.q_stops[self._get_state(), destination]

    def _get_xy(self):
        return self.xy[self._get_state()]



C:\Users\fabian.woellenweber\AppData\Local\Temp\ipykernel_40488\1267304800.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-dark")


In [78]:
episodes = 10 

env = TSPEnvironment(episodes)


for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    Loops = 0
    

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
        Loops += 1
    print('Epsiode: {} Score: {} Episodes: {}'.format(episode,score.round(2),Loops))

TSP-Environment initialized with 10 random stops
Starting Point: [2]
Visited Stops: [8]
Visited Stops: [8, 6]
Visited Stops: [8, 6, 7]
Visited Stops: [8, 6, 7, 0]
Visited Stops: [8, 6, 7, 0]
Visited Stops: [8, 6, 7, 0, 3]
Visited Stops: [8, 6, 7, 0, 3, 1]
Visited Stops: [8, 6, 7, 0, 3, 1]
Visited Stops: [8, 6, 7, 0, 3, 1]
Visited Stops: [8, 6, 7, 0, 3, 1, 5]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4]
Visited Stops: [8, 6, 7, 0, 3, 1, 5, 9, 4, 2]
Epsiode: 1 Score: -424.01 Episodes: 18
Visited Stops: [5, 3]
Visited Stops: [5, 3, 6]
Visited Stops: [5, 3, 6, 8]
Visited Stops: [5, 3, 6, 8, 1]
Visited Stops: [5, 3, 6, 8, 1]
Visited Stops: [5, 3, 6, 8, 1]
Visited Stops: [5, 3, 6, 8, 1, 7]
Visited Stops: [5, 3, 6, 8, 1, 7]
Visited Stops: [5, 3, 6

In [81]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent, DDPGAgent, ContinuousDQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.layers import Lambda


# Learning factors...
EPISODES = 50
discount_factor = 0.99
learning_rate = 0.001
epsilon_start = 1.0
epsilon_decay_steps = 0.999
epsilon_end = 0.01
batch_size = 64
train_start = 1000

# create replay memory using deque
#memory = deque(maxlen=2000)

# Create gym environment
env = TSPEnvironment(EPISODES)

state_size = env.observation_space.n
action_size = env.action_space.n

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.mean_squared_error

# Create a replay buffer to store transitions
replay_buffer = []

# Define the update frequency and batch size
update_frequency = 10
batch_size = 10

# The following function creates a neural network which is used as an 
# approximate Q function
# Input: state 
# Output: Q Value of each action
def build_model(state_size, action_size):
    model = Sequential()
    model.get_config()
    model.add(Dense(32, input_dim=state_size, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

# Create the TSP agent
def build_agent(model, action_size):
    # Use Epsilon-Greedy policy for exploration
    policy = EpsGreedyQPolicy()
    # Create memory for storing transitions
    memory = SequentialMemory(limit=50000, window_length=1)
    # Create the DQN agent
    agent = ContinuousDQNAgent(nb_actions=action_size, memory=memory, nb_steps_warmup=10,
                               V_model=None, L_model=None, mu_model=None)
    
    return agent


# Create the TSP model
model = build_model(state_size, action_size)
# Create the TSP agent
agent = build_agent(model, action_size)
# Train the agent
agent.compile(Adam(lr=1e-3), metrics=['mae'])


TSP-Environment initialized with 50 random stops
Starting Point: [35]


AttributeError: 'NoneType' object has no attribute 'get_config'

c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


AttributeError: 'NoneType' object has no attribute 'get_config'

In [58]:
import tensorflow as tf
from tensorflow.keras.models import Sequential

## Initialize important variables
env_name = 'TSPEnv'
bRender = False
load_model = False

# Learning factors...
EPISODES = 99
discount_factor = 0.99
learning_rate = 0.001
epsilon_start = 1.0
epsilon_decay_steps = 0.999
epsilon_end = 0.01
batch_size = 64
train_start = 1000

# create replay memory using deque
#memory = deque(maxlen=2000)

# Create gym environment
env = TSPEnvironment(EPISODES)

state_size = env.observation_space.n
action_size = env.action_space.n

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.mean_squared_error

# Create a replay buffer to store transitions
replay_buffer = []

# Define the update frequency and batch size
update_frequency = 10
batch_size = 10

# The following function creates a neural network which is used as an 
# approximate Q function
# Input: state 
# Output: Q Value of each action
def build_model():
    model = Sequential()
    model.add(Dense(32, input_dim=state_size, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(action_size, activation='linear', kernel_initializer='he_uniform'))
    model.summary()
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

# after some time interval update the target model to be same with model
# This is done using the following function
def update_target_model():
    target_model.set_weights(model.get_weights())

# create main model and target model
model = build_model()
target_model = build_model()

# We call that function in the beginning to make sure model and target_model have the same weights initally
update_target_model()

# Run the TSP loop
num_episodes = 50
for episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()

    # Calculate the epsilon for this episode
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay_steps)

    while True:
        # Select an action using the DQN model and an epsilon-greedy policy
        if np.random.rand() < epsilon:
            # Explore: choose a random action
            action = np.random.randint(env.action_space.n)
        else:
            # Exploit: choose the action with the highest Q-value
            action = np.argmax(model(np.expand_dims(state, axis=0)).numpy())

        # Take the action and observe the reward and next state
        next_state, reward, done, _ = env.step(action)

        # Store the transition in the replay buffer
        replay_buffer.append((state, action, reward, next_state))

        # If the replay buffer has reached a certain size, sample a batch of transitions and update the model
        if len(replay_buffer) >= batch_size:
            # Sample a batch of transitions
            transitions = np.random.sample(replay_buffer,batch_size)
            states, actions, rewards, next_states = zip(*transitions)

            # Convert the states and next_states to tensors
            states = tf.convert_to_tensor(states, dtype=tf.float32)
            next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
            
            print(f'States: {states}')
            print(f'States: {next_states}')

            # Compute the Q-values for the current states and the next states
            current_q_values = model(states)
            next_q_values = target_model(next_states)

            # Compute the expected Q-values
            expected_q_values = rewards + np.max(next_q_values.numpy(), axis=1)

            # Compute the loss
            loss = loss_fn(expected_q_values, current_q_values)

            # Update the model
            optimizer.minimize(loss, model.trainable_variables)

            # Clear the replay buffer
            replay_buffer = []

        # Update the target model every `update_frequency` steps
        if episode % update_frequency == 0:
            update_target_model()

        # Update the state
        state = next_state

        # If the episode is done, break out of the inner loop
        if done:
            break



    


TSP-Environment initialized with 99 random stops
Starting Point: [50]
Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_114 (Dense)           (None, 32)                3200      
                                                                 
 dense_115 (Dense)           (None, 32)                1056      
                                                                 
 dense_116 (Dense)           (None, 99)                3267      
                                                                 
Total params: 7,523
Trainable params: 7,523
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 32)                3200      
                        

TypeError: object of type 'int' has no len()